In [1]:
!pip install textattack==0.3.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 26.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 517.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
INFO: This is taking longer than usual. You might need

In [2]:
!pip install transformers torch sentencepiece

  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
Using cached nvidia_cublas_

In [3]:
# 3. JSON 데이터 업로드
from google.colab import files
import json

uploaded = files.upload()

with open("math.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

Saving math.json to math.json


In [4]:
# 라이브러리 import
import random
import json
import requests
import time
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from google.colab import files

# 1. Papago API 설정
client_id = '64yuz72rjr'
client_secret = 'mpxNeA4kw4gIEe2TgfRDfnWSzeWBJ1Sz0FfLkUDW'

def papago_translate(text, source_lang, target_lang):
    url = "https://naveropenapi.apigw.ntruss.com/nmt/v1/translation"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret
    }
    data = {
        "source": source_lang,
        "target": target_lang,
        "text": text
    }
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json()['message']['result']['translatedText']
    else:
        print(f"번역 오류 발생: {response.status_code}, 응답: {response.text}")
        return None

# 2. HuggingFace T5 Paraphraser 로딩
model_name = "ramsrigouthamg/t5_paraphraser"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
paraphraser = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

def generate_paraphrases(text, num_return_sequences=3):
    input_text = f"paraphrase: {text} </s>"
    outputs = paraphraser(
        input_text,
        max_length=100,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=100,
        top_p=0.92,
        temperature=0.8
    )
    return [output['generated_text'] for output in outputs]

# 3. Synonym 치환
synonym_dict = {
    # 명사
    "확률": ["가능성", "나올 비율"],
    "경우의 수": ["가능한 경우", "조합 수"],
    "자연수": ["양의 정수", "positive 정수"],
    "정사각형": ["모든 변이 같은 사각형", "네 변이 같은 사각형"],
    "정삼각형": ["모든 변이 같은 삼각형", "세 변이 같은 삼각형"],
    "수열": ["숫자 배열", "연속된 수"],
    "항": ["번째 수", "원소"],
    "둘레": ["외곽 길이", "주위 길이"],
    "넓이": ["면적", "표면적"],
    "부피": ["체적", "공간 크기"],
    "비율": ["비", "비례 관계"],
    "방정식": ["식", "등식 문제"],

    # 동사
    "구하시오": ["계산하시오", "찾으시오"],
    "뽑을 확률은": ["선택될 가능성은", "나올 확률은"],
    "더했더니": ["합했더니", "합계는"],
    "곱했더니": ["곱의 결과는", "곱한 결과는"],
    "일치하는 경우는": ["겹치는 횟수는", "같아지는 경우는"],

    # 형용사/부사
    "같은": ["동일한", "일치하는"],
    "서로 다른": ["중복되지 않는", "겹치지 않는"],
    "가장 큰": ["최대의", "제일 큰"],
    "최소": ["가장 작은", "가장 적은"],
}

def synonym_replace(korean_text):
    words = korean_text.split()
    new_words = []
    for word in words:
        replaced = False
        for key, synonyms in synonym_dict.items():
            if key in word:
                if random.random() < 0.5:
                    word = word.replace(key, random.choice(synonyms))
                    replaced = True
                    break
        new_words.append(word)
    return ' '.join(new_words)

# 4. Style Transfer
def style_transfer(text, apply_prob=0.6):
    if random.random() > apply_prob:
        return text

    endings = {
      # 문장 종결
      "무엇인가요": ["뭘까?", "뭐야?"],
      "무엇일까요": ["뭘까?", "뭐야?"],
      "몇 개일까요": ["몇 개야?", "몇 개나 있어?"],
      "몇 개인가요": ["몇 개야?", "얼마나 있어?"],
      "구하시오": ["구해봐.", "한번 계산해봐."],
      "값은?": ["값이 뭐야?", "얼마야?"],

      # 구조 전환
      "몇 번 겹치는가": ["몇 번 만나게 될까?", "몇 번 같이 있을까?"],
      "가능한 경우의 수는": ["몇 가지 방법이 있을까?", "가능한 조합은 몇 개야?"],
      "경우의 수는": ["몇 가지 경우가 있을까?", "몇 개나 될까?"],
      "확률은?": ["확률이 어떻게 돼?", "어느 정도 나올까?"],

      # 수식 전환
      "일 때, x의 값은?": ["라면 x는?", "이면 x는 뭘까?"],
      "일 때 x는?": ["라면 x는?", "이면 x는 뭐야?"]
    }

    for formal, casual_list in endings.items():
        if formal in text:
            text = text.replace(formal, random.choice(casual_list))
    return text

# 5. 전체 프로세스 함수
def full_back_translate_with_paraphrase_style(korean_text, num_augments=5):
    en_text = papago_translate(korean_text, source_lang='ko', target_lang='en')
    time.sleep(0.2)

    paraphrases = generate_paraphrases(en_text, num_return_sequences=num_augments)
    time.sleep(0.2)

    results = []
    for p in paraphrases:
        ko_text = papago_translate(p, source_lang='en', target_lang='ko')
        time.sleep(0.2)

        ko_text_synonym = synonym_replace(ko_text)
        final_text = style_transfer(ko_text_synonym)

        results.append(final_text)

    return results

# 7. 증강하기
augmented_data = []

for item in raw_data:
    question = item['Question']
    answer = item['Answer']

    # 질문을 여러 개 증강
    augmented_questions = full_back_translate_with_paraphrase_style(question, num_augments=5)

    for aug_q in augmented_questions:
        augmented_data.append({
            "Question": aug_q,
            "Answer": answer
        })

# 8. 증강된 데이터 저장
output_filename = "augmented_math.json"

with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(augmented_data, f, ensure_ascii=False, indent=4)

# 9. Colab에서 바로 다운로드
files.download(output_filename)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>